In [ ]:
pip install qiskit qiskit_aer qiskit-ibm-runtime qiskit[visualization]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 4.3 MB/s eta 0:00:00
  Using cached qiskit-1.4.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
INFO: pip is looking at multiple versions of qiskit[visualization] to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 14.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 46.4 MB/s eta 0:00:00
Using cached qiskit-1.4.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (6.7 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 343.3/343.3 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.9/434.9 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.4/69.4 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.

In [ ]:
from qiskit import QuantumCircuit, Aer, execute
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

# --- Step 1: Toy DisCoCat Embedding ---

def disco_cat_embedding(text):
    """
    A toy embedding that maps specific texts to computational basis states.
    In a realistic DisCoCat model, words (and their grammar) are mapped to
    quantum states via tensor contractions. Here, we use a simple dictionary.
    """
    mapping = {
        "Alice loves Bob": "00",
        "Alice hates Bob": "01",
        "Carol loves Bob": "10",
        "Bob loves Carol": "11"
    }
    return mapping.get(text, "00")

# Define a small memory of texts.
memory_texts = ["Alice loves Bob", "Alice hates Bob",
                "Carol loves Bob", "Bob loves Carol"]

# Define the query text.
query_text = "Alice loves Bob"

# Compute the toy embeddings.
embedded_states = {text: disco_cat_embedding(text) for text in memory_texts}
print("Embedded states:", embedded_states)
query_state = disco_cat_embedding(query_text)
print("Query state:", query_state)

# For this toy example, our target state is the one matching the query.
target_state = query_state  # e.g. "00" for "Alice loves Bob"

# --- Step 2: Build an Oracle to Mark the Target State ---

def build_oracle(target):
    """
    Build an oracle that flips the phase of the computational basis state
    corresponding to 'target' (a string like '00').
    The method uses X gates to convert the target state into |11...1>,
    applies a multi-controlled-Z (via a controlled Toffoli), then undoes the X gates.
    """
    num_qubits = len(target)
    qc = QuantumCircuit(num_qubits)
    # For each bit, if target bit is '0', flip it so that the target becomes all 1's.
    for i, bit in enumerate(target):
        if bit == '0':
            qc.x(i)
    # Apply controlled-Z; for 2 qubits, this is done with a Hadamard, MCX, then Hadamard.
    qc.h(num_qubits - 1)
    qc.mct(list(range(num_qubits - 1)), num_qubits - 1)  # multi-controlled X (Toffoli for 2 qubits)
    qc.h(num_qubits - 1)
    # Undo the initial X gates.
    for i, bit in enumerate(target):
        if bit == '0':
            qc.x(i)
    oracle_gate = qc.to_gate()
    oracle_gate.name = "Oracle"
    return oracle_gate

oracle_gate = build_oracle(target_state)

# --- Step 3: Grover's Search Circuit ---

num_qubits = 2
grover_circuit = QuantumCircuit(num_qubits)

# Start with equal superposition.
grover_circuit.h(range(num_qubits))

# Apply the oracle.
grover_circuit.append(oracle_gate, range(num_qubits))

# Apply the diffusion operator.
grover_circuit.h(range(num_qubits))
grover_circuit.x(range(num_qubits))
grover_circuit.h(num_qubits - 1)
grover_circuit.mct(list(range(num_qubits - 1)), num_qubits - 1)
grover_circuit.h(num_qubits - 1)
grover_circuit.x(range(num_qubits))
grover_circuit.h(range(num_qubits))

# Measure all qubits.
grover_circuit.measure_all()

print(grover_circuit.draw(output='text'))

# --- Step 4: Execute and Visualize ---

backend = Aer.get_backend('qasm_simulator')
result = execute(grover_circuit, backend, shots=1024).result()
counts = result.get_counts()
print("Grover search results:", counts)
plot_histogram(counts)
plt.show()


In [ ]:
# Advanced Pipeline: From Professional Corpus to Quantum Retrieval via Grover's Search

# Step 0. Imports and setup
from lambeq import BobcatParser, IQPAnsatz, remove_cups
from qiskit import QuantumCircuit, Aer, execute
from qiskit.quantum_info import Statevector
import numpy as np
import math

# Set random seed for reproducibility
np.random.seed(42)

# Step 1. Load a (simulated) professional corpus.
# In practice, this could be loaded from a CSV, database, or API.
corpus = [
    "The stock market rallied after the announcement of positive earnings.",
    "The government introduced new environmental policies to combat climate change.",
    "Breakthroughs in quantum computing are expected to revolutionize cryptography.",
    "Advances in renewable energy technologies promise to reduce carbon emissions.",
    "Innovations in artificial intelligence are transforming the healthcare industry.",
    "Global trade is impacted by fluctuating geopolitical tensions and tariffs.",
    "Researchers unveiled a novel method for data encryption using quantum principles.",
    "The central bank raised interest rates to control inflation amidst economic growth."
]

# Define a query sentence (e.g., looking for quantum-inspired finance news)
query_sentence = "Quantum computing techniques are reshaping encryption and finance."

# Step 2. Convert each sentence to a string diagram using lambeq's BobcatParser.
# The BobcatParser parses the sentence into a diagram reflecting its grammatical structure.
parser = BobcatParser(verbose='none')
corpus_diagrams = parser.sentences2diagrams(corpus)
query_diagram = parser.sentence2diagram(query_sentence)

# Optionally simplify diagrams (e.g. by removing cups)
corpus_diagrams = [remove_cups(diag) for diag in corpus_diagrams]
query_diagram = remove_cups(query_diagram)

# Step 3. Convert diagrams to quantum circuits using a parameterization (e.g. IQPAnsatz).
# The IQPAnsatz is one way to map the DisCoCat diagram into a quantum circuit.
# (In a full application, the number of qubits and ansatz details would be tuned to the embedding dimensions.)
ansatz = IQPAnsatz(n_qubits=4)  # Example: use 4 qubits per sentence
corpus_circuits = [ansatz(diag) for diag in corpus_diagrams]
query_circuit = ansatz(query_diagram)

# Step 4. Simulate the circuits to obtain quantum statevectors.
simulator_sv = Aer.get_backend('statevector_simulator')

def get_statevector(qc: QuantumCircuit):
    job = execute(qc, backend=simulator_sv)
    result = job.result()
    return result.get_statevector(qc)

corpus_states = [get_statevector(qc) for qc in corpus_circuits]
query_state = get_statevector(query_circuit)

# Step 5. Define a similarity metric (fidelity) between two statevectors.
def fidelity(state1, state2):
    # Fidelity is |<state1|state2>|^2
    return np.abs(np.vdot(state1, state2))**2

# Compute the similarity (fidelity) between the query and each document in the corpus.
similarities = [fidelity(query_state, doc_state) for doc_state in corpus_states]
print("Fidelity similarities between query and corpus:")
for idx, sim in enumerate(similarities):
    print(f"Doc {idx}: {sim:.4f}")

# Step 6. Classical retrieval: Identify best match based on maximum fidelity.
best_index_classical = np.argmax(similarities)
print("\nClassical retrieval result:")
print(f"Best matching sentence (index {best_index_classical}):\n  {corpus[best_index_classical]}")

# Step 7. Quantum retrieval using Grover's search.
# We assume a threshold similarity for marking a document as a "target".
# (In practice, one might use iterative thresholding or more sophisticated oracles.)
threshold = 0.7
print("\nUsing similarity threshold:", threshold)
marked_indices = [i for i, sim in enumerate(similarities) if sim >= threshold]
print("Marked indices based on threshold:", marked_indices)

# For Grover search, we need an index register that represents document indices.
# With len(corpus)=8, we need n_qubits_index = ceil(log2(8)) = 3.
n_index_qubits = math.ceil(math.log2(len(corpus)))
print(f"Using {n_index_qubits} qubits to represent document indices.")

# Build a Grover oracle that flips the phase of basis states corresponding to marked indices.
def build_oracle(n_qubits: int, marked_indices: list) -> QuantumCircuit:
    qc = QuantumCircuit(n_qubits)
    size = 2**n_qubits
    # Create a diagonal operator with -1 for marked indices and +1 for others.
    diag = [1] * size
    for i in marked_indices:
        diag[i] = -1
    # Apply the diagonal operator.
    qc.diagonal(np.array(diag), list(range(n_qubits)))
    return qc.to_gate(label="Oracle")

oracle_gate = build_oracle(n_index_qubits, marked_indices)

# Grover diffuser (inversion about the mean)
def grover_diffuser(n_qubits: int) -> QuantumCircuit:
    qc = QuantumCircuit(n_qubits)
    qc.h(range(n_qubits))
    qc.x(range(n_qubits))
    qc.h(n_qubits - 1)
    qc.mct(list(range(n_qubits - 1)), n_qubits - 1)  # multi-controlled Toffoli (assumes availability)
    qc.h(n_qubits - 1)
    qc.x(range(n_qubits))
    qc.h(range(n_qubits))
    return qc.to_gate(label="Diffuser")

diffuser_gate = grover_diffuser(n_index_qubits)

# Determine the number of Grover iterations (approximately floor(pi/4 * sqrt(N/M))).
M = len(marked_indices) if marked_indices else 1
iterations = int(np.floor((math.pi/4) * math.sqrt(2**n_index_qubits / M)))
print("Number of Grover iterations:", iterations)

# Construct the Grover search circuit on the index register.
grover_circuit = QuantumCircuit(n_index_qubits)
grover_circuit.h(range(n_index_qubits))  # initial uniform superposition

for _ in range(iterations):
    grover_circuit.append(oracle_gate, range(n_index_qubits))
    grover_circuit.append(diffuser_gate, range(n_index_qubits))

grover_circuit.measure_all()
print("\nGrover search circuit:")
print(grover_circuit.draw(output='text'))

# Step 8. Run the Grover search circuit using Qiskit's qasm_simulator.
qasm_backend = Aer.get_backend('qasm_simulator')
grover_result = execute(grover_circuit, backend=qasm_backend, shots=1024).result()
grover_counts = grover_result.get_counts()
print("\nGrover search measurement results:")
print(grover_counts)

# Interpret the measurement: the most frequent outcome is the retrieved index.
retrieved_index = int(max(grover_counts, key=grover_counts.get), 2)
print("\nRetrieved index from Grover search:", retrieved_index)
print("Best matching sentence (Grover search):")
print(corpus[retrieved_index])
